# Example: Levels

In this examples we describe the concept of `levels` in BFV and CKKS and the
related objects that represent them in Microsoft SEAL.

In Microsoft SEAL a set of encryption parameters (excluding the random number
generator) is identified uniquely by a 256-bit hash of the parameters. This
hash is called the `parms_id` and can be easily accessed and printed at any
time. The hash will change as soon as any of the parameters is changed.

When a SEALContext is created from a given EncryptionParameters instance,
Microsoft SEAL automatically creates a so-called `modulus switching chain`,
which is a chain of other encryption parameters derived from the original set.
The parameters in the modulus switching chain are the same as the original
parameters with the exception that size of the coefficient modulus is
decreasing going down the chain. More precisely, each parameter set in the
chain attempts to remove the last coefficient modulus prime from the
previous set; this continues until the parameter set is no longer valid
(e.g., `plain_modulus` is larger than the remaining `coeff_modulus`). It is easy
to walk through the chain and access all the parameter sets. Additionally,
each parameter set in the chain has a `chain index` that indicates its
position in the chain so that the last set has index 0. We say that a set
of encryption parameters, or an object carrying those encryption parameters,
is at a higher level in the chain than another set of parameters if its the
chain index is bigger, i.e., it is earlier in the chain.

Each set of parameters in the chain involves unique pre-computations performed
when the SEALContext is created, and stored in a `SEALContext.ContextData`
object. The chain is basically a linked list of `SEALContext.ContextData`
objects, and can easily be accessed through the `SEALContext` at any time. Each
node can be identified by the parms_id of its specific encryption parameters
(`poly_modulus_degree` remains the same but `coeff_modulus` varies).

In [1]:
import seal
import example_helper

parms = seal.EncryptionParameters(seal.scheme_type.bfv)
poly_modulus_degree = 8192
parms.poly_modulus_degree = poly_modulus_degree

In this example we use a custom coeff_modulus, consisting of 5 primes of
sizes 50, 30, 30, 50, and 50 bits. Note that this is still OK according to
the explanation in `1_bfv_basics.ipynb`. Indeed, `CoeffModulus.MaxBitCount(poly_modulus_degree)` returns 218 (greater than 50+30+30+50+50=210).

Due to the modulus switching chain, the order of the 5 primes is significant.
The last prime has a special meaning and we call it the `special prime`. Thus,
the first parameter set in the modulus switching chain is the only one that
involves the special prime. All key objects, such as `SecretKey`, are created
at this highest level. All data objects, such as Ciphertext, can be only at
lower levels. The special prime should be as large as the largest of the
other primes in the `coeff_modulus`, although this is not a strict requirement.

                special prime +---------+
                                        |
                                        v
    coeff_modulus: { 50, 30, 30, 50, 50 }  +---+  Level 4 (all keys; `key level`)
                                                |
                                                |
        coeff_modulus: { 50, 30, 30, 50 }  +---+  Level 3 (highest `data level`)
                                                |
                                                |
            coeff_modulus: { 50, 30, 30 }  +---+  Level 2
                                                |
                                                |
                coeff_modulus: { 50, 30 }  +---+  Level 1
                                                |
                                                |
                    coeff_modulus: { 50 }  +---+  Level 0 (lowest level)

In [2]:
parms.coeff_modulus = seal.CoeffModulus.Create(poly_modulus_degree, [50,30,30,50,50])

In this example the plain_modulus does not play much of a role; we choose
some reasonable value.

In [3]:
parms.plain_modulus = seal.PlainModulus.Batching(poly_modulus_degree, 20)
context = seal.SEALContext(parms)
example_helper.print_parameters(context)

/
| Encryption parameters
| scheme: bfv
| poly_modulus_degree: 8192
| coeff_modulus size: 210(50 + 30 + 30 + 50 + 50) bits
| plain_modulus: 1032193
\


There are convenience method for accessing the `SEALContext.ContextData` for
some of the most important levels:

  - `SEALContext.key_context_data`: access to key level ContextData
  - `SEALContext.first_context_data`: access to highest data level ContextData
  - `SEALContext.last_context_data`: access to lowest level ContextData

We iterate over the chain and print the `parms_id` for each set of parameters.

In [4]:
print("Print the modulus switching chain.")

# First print the key level parameter information.
context_data = context.key_context_data
print(f"----> Level (chain index): {context_data.chain_index}", end="")
print(f" ...... key_context_data")
print(f"      parms_id: {context_data.parms_id}")
print(f"      coeff_modulus primes: ", end='')
for prime in context_data.parms.coeff_modulus:
    print(f"{hex(prime.value)} ", end="")
print()
print("\\")
print(" \\-->", end="")

# Next iterate over the remaining (data) levels.
context_data = context.first_context_data
while context_data:
    print(f" Level (chain index): {context_data.chain_index}", end="")
    if context_data.parms_id == context.first_parms_id:
        print(f" ...... first_context_data")
    elif context_data.parms_id == context.last_parms_id:
        print(f" ...... last_context_data")
    else:
        print()
    print(f"      parms_id: {context_data.parms_id}")
    print(f"      coeff_modulus primes: ", end='')
    for prime in context_data.parms.coeff_modulus:
        print(f"{hex(prime.value)} ", end="")
    print()
    print("\\")
    print(" \\-->", end="")
    context_data = context_data.next_context_data
print(" End of chain reached")

Print the modulus switching chain.
----> Level (chain index): 4 ...... key_context_data
      parms_id: [2796926214238341906, 7385196832706630708, 1778331432907072121, 9574839751679865602]
      coeff_modulus primes: 0x3ffffffef4001 0x3ffe8001 0x3fff4001 0x3fffffffcc001 0x3ffffffffc001 
\
 \--> Level (chain index): 3 ...... first_context_data
      parms_id: [2386594185047272216, 3177129986462177089, 5264335169243394227, 3608211254040463884]
      coeff_modulus primes: 0x3ffffffef4001 0x3ffe8001 0x3fff4001 0x3fffffffcc001 
\
 \--> Level (chain index): 2
      parms_id: [9611362035343820607, 16250750482064473934, 11666325943188447289, 11197906540159540193]
      coeff_modulus primes: 0x3ffffffef4001 0x3ffe8001 0x3fff4001 
\
 \--> Level (chain index): 1
      parms_id: [8338375294373729721, 6255090713888968186, 9221042086196212239, 13262945982911515329]
      coeff_modulus primes: 0x3ffffffef4001 0x3ffe8001 
\
 \--> Level (chain index): 0 ...... last_context_data
      parms_id: [1264594

We create some keys and check that indeed they appear at the highest level.

In [5]:
keygen = seal.KeyGenerator(context)
secret_key = keygen.secret_key
public_key = keygen.create_public_key()
relin_keys = keygen.create_relin_keys()

print(f"Print the parameter IDs of generated elements.")
print(f"    + public_key: {public_key.parms_id}")
print(f"    + secret_key: {secret_key.parms_id}")
print(f"    + relin_keys: {relin_keys.parms_id}")

encryptor = seal.Encryptor(context, public_key)
evaluator = seal.Evaluator(context)
decryptor = seal.Decryptor(context, secret_key)

Print the parameter IDs of generated elements.
    + public_key: [2796926214238341906, 7385196832706630708, 1778331432907072121, 9574839751679865602]
    + secret_key: [2796926214238341906, 7385196832706630708, 1778331432907072121, 9574839751679865602]
    + relin_keys: [2796926214238341906, 7385196832706630708, 1778331432907072121, 9574839751679865602]


In the BFV scheme plaintexts do not carry a `parms_id`, but ciphertexts do. Note
how the freshly encrypted ciphertext is at the highest data level.

In [6]:
plain = seal.Plaintext("1x^3 + 2x^2 + 3x^1 + 4")
encrypted = encryptor.encrypt(plain)
print(f"    + plain:     {plain.parms_id} (not set in BFV)")
print(f"    + encrypted: {encrypted.parms_id}")

    + plain:     [0, 0, 0, 0] (not set in BFV)
    + encrypted: [2386594185047272216, 3177129986462177089, 5264335169243394227, 3608211254040463884]


`Modulus switching` is a technique of changing the ciphertext parameters down
in the chain. The function `Evaluator.mod_switch_to_next` always switches to
the next level down the chain, whereas `Evaluator.mod_switch_to` switches to
a parameter set down the chain corresponding to a given `parms_id`. However, it
is impossible to switch up in the chain.

In [7]:
print("Perform modulus switching on encrypted and print.")

context_data = context.first_context_data
while context_data.next_context_data:
    print(f" Level (chain index): {context_data.chain_index}")
    print(f"      parms_id of encrypted: {encrypted.parms_id}")
    print(f"      noise budget at this level: {decryptor.invariant_noise_budget(encrypted)} bits")
    print("\\")
    print(" \\-->", end="")
    evaluator.mod_switch_to_next_inplace(encrypted)
    context_data = context_data.next_context_data
print(f" Level (chain index): {context_data.chain_index}", end="")
print(f"      parms_id of encrypted: {encrypted.parms_id}")
print(f"      noise budget at this level: {decryptor.invariant_noise_budget(encrypted)} bits")
print("\\")
print(" \\-->", end="")
print(" End of chain reached")

Perform modulus switching on encrypted and print.
 Level (chain index): 3
      parms_id of encrypted: [2386594185047272216, 3177129986462177089, 5264335169243394227, 3608211254040463884]
      noise budget at this level: 132 bits
\
 \--> Level (chain index): 2
      parms_id of encrypted: [9611362035343820607, 16250750482064473934, 11666325943188447289, 11197906540159540193]
      noise budget at this level: 82 bits
\
 \--> Level (chain index): 1
      parms_id of encrypted: [8338375294373729721, 6255090713888968186, 9221042086196212239, 13262945982911515329]
      noise budget at this level: 52 bits
\
 \--> Level (chain index): 0      parms_id of encrypted: [12645946865612918007, 3410116064097512307, 265341692546732382, 11895432757488484810]
      noise budget at this level: 22 bits
\
 \--> End of chain reached


At this point it is hard to see any benefit in doing this: we lost a huge
amount of noise budget (i.e., computational power) at each switch and seemed
to get nothing in return. Decryption still works.

In [8]:
print("Decrypt still works after modulus switching.")
plain = decryptor.decrypt(encrypted)
print(f"    + Decryption of encrypted: {plain.to_string()}")

Decrypt still works after modulus switching.
    + Decryption of encrypted: 1x^3 + 2x^2 + 3x^1 + 4


However, there is a hidden benefit: the size of the ciphertext depends
linearly on the number of primes in the coefficient modulus. Thus, if there
is no need or intention to perform any further computations on a given
ciphertext, we might as well switch it down to the smallest (last) set of
parameters in the chain before sending it back to the secret key holder for
decryption.

Also the lost noise budget is actually not an issue at all, if we do things
right, as we will see below.

First we recreate the original ciphertext and perform some computations.

In [9]:
print("Computation is more efficient with modulus switching.")

print("Compute the 8th power.")
encrypted = encryptor.encrypt(plain)
print(f"    + Noise budget fresh:                   {decryptor.invariant_noise_budget(encrypted)} bits")
evaluator.square_inplace(encrypted)
evaluator.relinearize_inplace(encrypted, relin_keys)
print(f"    + Noise budget of the 2nd power:        {decryptor.invariant_noise_budget(encrypted)} bits")
evaluator.square_inplace(encrypted)
evaluator.relinearize_inplace(encrypted, relin_keys)
print(f"    + Noise budget of the 4th power:        {decryptor.invariant_noise_budget(encrypted)} bits")

# Surprisingly, in this case modulus switching has no effect at all on the noise budget.
evaluator.mod_switch_to_next_inplace(encrypted)
print(f"    + Noise budget after modulus switching: {decryptor.invariant_noise_budget(encrypted)} bits")

Computation is more efficient with modulus switching.
Compute the 8th power.
    + Noise budget fresh:                   132 bits
    + Noise budget of the 2nd power:        100 bits
    + Noise budget of the 4th power:        67 bits
    + Noise budget after modulus switching: 67 bits


This means that there is no harm at all in dropping some of the coefficient
modulus after doing enough computations. In some cases one might want to
switch to a lower level slightly earlier, actually sacrificing some of the
noise budget in the process, to gain computational performance from having
smaller parameters. We see from the print-out that the next modulus switch
should be done ideally when the noise budget is down to around 25 bits.

In [10]:
evaluator.square_inplace(encrypted)
evaluator.relinearize_inplace(encrypted, relin_keys)
print(f"    + Noise budget of the 8th power:        {decryptor.invariant_noise_budget(encrypted)} bits")
evaluator.mod_switch_to_next_inplace(encrypted)
print(f"    + Noise budget after modulus switching: {decryptor.invariant_noise_budget(encrypted)} bits")

    + Noise budget of the 8th power:        35 bits
    + Noise budget after modulus switching: 35 bits


At this point the ciphertext still decrypts correctly, has very small size,
and the computation was as efficient as possible. Note that the decryptor
can be used to decrypt a ciphertext at any level in the modulus switching
chain.

In [11]:
plain = decryptor.decrypt(encrypted)
print(f"    + Decryption of the 8th power (hexadecimal)")
print(f"    {plain.to_string()}")

    + Decryption of the 8th power (hexadecimal)
    1x^24 + 10x^23 + 88x^22 + 330x^21 + EFCx^20 + 3A30x^19 + C0B8x^18 + 22BB0x^17 + 58666x^16 + C88D0x^15 + 9C377x^14 + F4C0Ex^13 + E8B38x^12 + 5EE89x^11 + F8BFFx^10 + 30304x^9 + 5B9D4x^8 + 12653x^7 + 4DFB5x^6 + 879F8x^5 + 825FBx^4 + F1FFEx^3 + 3FFFFx^2 + 60000x^1 + 10000


In BFV modulus switching is not necessary and in some cases the user might
not want to create the modulus switching chain, except for the highest two
levels. This can be done by passing a bool `false` to SEALContext constructor.

We can check that indeed the modulus switching chain has been created only
for the highest two levels (key level and highest data level). The following
loop should execute only once.

In [12]:
context = seal.SEALContext(parms, False)

print("Optionally disable modulus switching chain expansion.")
print("Print the modulus switching chain.")
print("---->", end="")
context_data = context.key_context_data
while context_data:
    print(f" Level (chain index): {context_data.chain_index}")
    print(f"      parms_id: {context_data.parms_id}")
    print(f"      coeff_modulus primes: ", end='')
    for prime in context_data.parms.coeff_modulus:
        print(f"{hex(prime.value)} ", end="")
    print()
    print("\\")
    print(" \\-->", end="")
    context_data = context_data.next_context_data
print(" End of chain reached")

Optionally disable modulus switching chain expansion.
Print the modulus switching chain.
----> Level (chain index): 1
      parms_id: [2796926214238341906, 7385196832706630708, 1778331432907072121, 9574839751679865602]
      coeff_modulus primes: 0x3ffffffef4001 0x3ffe8001 0x3fff4001 0x3fffffffcc001 0x3ffffffffc001 
\
 \--> Level (chain index): 0
      parms_id: [2386594185047272216, 3177129986462177089, 5264335169243394227, 3608211254040463884]
      coeff_modulus primes: 0x3ffffffef4001 0x3ffe8001 0x3fff4001 0x3fffffffcc001 
\
 \--> End of chain reached


It is very important to understand how this example works since in the BGV
scheme modulus switching has a much more fundamental purpose and the next
examples will be difficult to understand unless these basic properties are
totally clear.